In [ ]:
from pandas import read_csv
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#pip install -U scikit-learn


In [ ]:
np.random.seed(20773)

pudahuel = pd.read_csv('Pudahuel Data/pudahuel-air-quality.csv', engine='python',
                       na_values=[' '])
pudahuel.replace('', np.nan, inplace=True)

pudahuel[' pm10'] = pudahuel[' pm10'].interpolate()
pudahuel[' o3'] = pudahuel[' o3'].interpolate()
pudahuel[' o3'] = pudahuel[' o3'] / 1000
pudahuel[' so2'] = pudahuel[' so2'].interpolate(method='linear', limit_direction='both')
pudahuel['date'] = pd.to_datetime(pudahuel['date']).dt.date
pudahuel.sort_values('date', inplace=True, ascending=False)
pudahuel.columns = ['date', 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co']

print(pudahuel['so2'])


In [ ]:
def juntar_datos_csv(nombre_carpeta, arreglo_columnas):
    # Obtiene la lista de todos los archivos que coinciden con el patrón
    csv_files = glob.glob(nombre_carpeta + "/*.csv")
    # Crea una lista para almacenar los DataFrames
    dfs = []

    # Lee cada archivo CSV y lo agrega a la lista de DataFrames
    for csv_file in csv_files:
        df = pd.read_csv(csv_file, engine='python', sep=';',
                         usecols=arreglo_columnas)
        dfs.append(df)

    # Combina los DataFrames en uno solo
    return pd.concat(dfs)


# Ejemplo de uso de la función juntar_datos_csv
agua_caida = juntar_datos_csv("Pudahuel Data/Agua Caida", [3, 4])
presion_humedad = juntar_datos_csv("Pudahuel Data/Presion y Humedad",
                                   list(range(3, 10)))
radiacion = juntar_datos_csv("Pudahuel Data/Radiacion", [3, 4])
viento = juntar_datos_csv("Pudahuel Data/Viento", [3, 5, 6, 7, 8, 9])



## Formateo de data frames a datos por día

Para datos de Agua Caída

In [ ]:
print(agua_caida)
agua_caida['momento'] = pd.to_datetime(agua_caida['momento'])
# Agregar una columna "date" que contenga solo la fecha (sin la hora)
agua_caida['date'] = agua_caida['momento'].dt.date
agua_caida_f = agua_caida.groupby('date')['rrInst'].sum()

Para datos de Radiación

In [ ]:
print(radiacion)
radiacion['momento'] = pd.to_datetime(radiacion['momento'])
# Agregar una columna "date" que contenga solo la fecha (sin la hora)
radiacion['date'] = radiacion['momento'].dt.date
# Obtener los valores máximos para cada día en las columnas especificadas
radiacion_f = radiacion.groupby('date')['radiacionGlobalInst'].max()
print(radiacion_f)

Para datos del Viento

In [ ]:
#print(viento)
viento['momento'] = pd.to_datetime(viento['momento'])
# Agregar una columna "date" que contenga solo la fecha (sin la hora)
viento['date'] = viento['momento'].dt.date
# Obtener los valores máximos para cada día en las columnas especificadas
#print(viento)
#print(viento.groupby('date').max())
viento_f = viento.groupby('date')[['ffInst', 'dd02Minutos', 'ff02Minutos',
                                   'dd10Minutos', 'ff10Minutos']].max()
print(viento_f)


In [ ]:
presion_humedad['momento'] = pd.to_datetime(presion_humedad['momento'])
# Agregar una columna "date" que contenga solo la fecha (sin la hora)
presion_humedad['date'] = presion_humedad['momento'].dt.date
# Obtener los valores máximos para cada día en las columnas especificadas
presion_humedad_f = presion_humedad.groupby('date')[['hr', 'p0', 'qfe1', 'qfe2', 'qff']]\
                                .mean().reset_index()
presion_humedad_f['qfe2'] = presion_humedad_f['qfe2'].interpolate(
    method='linear', limit_direction='backward', axis=0)
print(presion_humedad_f)


## Construccion Data Frame con todos los datos por fecha


In [ ]:
final_df = pudahuel.merge(viento_f, on='date')
final_df = final_df.merge(presion_humedad_f, on='date')
final_df = final_df.merge(agua_caida_f, on='date')
final_df = final_df.merge(radiacion_f, on='date')
print(final_df)
print(final_df.columns)



Los rangos y ponderaciones utilizados en el cálculo del Índice de Calidad del Aire de Santiago (ICAS) son definidos por la normativa chilena y están diseñados para evaluar la concentración de contaminantes atmosféricos y asignarles un valor numérico representativo de su nivel de riesgo para la salud humana y el medio ambiente.

Los rangos representan los límites o umbrales establecidos para cada contaminante. Por ejemplo, para el PM2.5, los rangos indican los intervalos de concentración que van desde niveles bajos hasta niveles peligrosos. Estos rangos pueden variar según el contaminante y la normativa vigente.

Las ponderaciones son los valores asignados a cada rango de concentración para determinar la contribución de cada contaminante al índice final. Estas ponderaciones suelen reflejar la importancia relativa de cada contaminante en términos de su impacto en la salud humana y el medio ambiente. Por ejemplo, se pueden asignar ponderaciones más altas a los contaminantes que tienen un mayor impacto en la calidad del aire y en la salud de las personas.

In [ ]:
# dataset = dataframe.values
# dataset = dataset.astype('float32')
# numpy.shape(dataset)
print(final_df.columns)


# Función para calcular el AQI
def calcular_ica_polucion(cp, pollutant):
    # Definir los puntos de interrupción y los rangos de AQI
    breakpoints = {
        'o3': [(0, 0.054), (0.055, 0.070), (0.071, 0.085), (0.086, 0.105),
                (0.106, 0.200), (0.201, 0.504)],
        'pm25': [(0, 12.0), (12.1, 35.4), (35.5, 55.4), (55.5, 150.4),
                 (150.5, 250.4), (250.5, 500.4)],
        'pm10': [(0, 54), (55, 154), (155, 254), (255, 354), (355, 424),
                 (425, 604)],
        'co': [(0, 4.4), (4.5, 9.4), (9.5, 12.4), (12.5, 15.4),
               (15.5, 30.4), (30.5, 50.4)],
        'so2': [(0, 35), (36, 75), (76, 185), (186, 304), (305, 604),
                (605, 1004)],
        'no2': [(0, 53), (54, 100), (101, 360), (361, 649),
                (650, 1249), (1250, 2049)]
    }
    rangos_ica = [(0, 50), (51, 100), (101, 150), (151, 200),
                  (201, 300), (301, 500)]
    for i, (bp_lo, bp_hi) in enumerate(breakpoints[pollutant]):
        if bp_lo <= cp <= bp_hi:
            i_lo, i_hi = rangos_ica[i]
            ip = ((i_hi - i_lo) / (bp_hi - bp_lo)) * (cp - bp_lo) + i_lo
            return round(ip)
    return np.nan


def calcular_polucion(df):
    # Calcular el AQI para cada contaminante y cada fila
    for pollutant in ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']:
        df[f'{pollutant}_ica'] = df[pollutant].apply(
            calcular_ica_polucion, pollutant=pollutant)

    # Calcular el AQI para cada fila
    df['ICA'] = df[[f'{pollutant}_ica'
                                for pollutant in ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]].max(axis=1)
    return df


final_df = calcular_polucion(final_df)

# Supongamos que tienes un DataFrame llamado df que contiene los datos de pm25, pm10, o3, no2 y co

test = 10

print(final_df.query("ICA > @test"))

In [ ]:
# Function to create the dataset
# Equation f(yt) = f(yt-5)
# Convert an array of values into a dataset matrix

# Convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg



### Funcion para desplazar en un intervalo los datos

In [ ]:
# Funcion que crea un dataframe con los valores desplazados en un
#  intervalo de tiempo de un data frame entregado
def create_interval_shifted_dataset(df, start, end, keep_column):
    df = df.sort_values(by='date')
    columns = df.columns.tolist()
    columns.remove(keep_column)
    columns.remove('date')

    dfs_auxiliares = {}
    for n in range(start, end + 1):
        dfs_auxiliares[n] = df.set_index('date').shift(n).reset_index()
    new_data = []
    for i, row in df.iterrows():
        current_date = row['date']
        new_row = {}

        for n in range(start, end + 1):
            df_auxiliar = dfs_auxiliares[n]
            df_filtrado = df_auxiliar[df_auxiliar['date'] == current_date]

            if not df_filtrado.empty:
                for column in columns:
                    column_name = f"{column}(t-{n})"
                    new_row[column_name] = df_filtrado.at[df_filtrado.index[0], column]

        if new_row:
            new_row[keep_column + "(t)"] = row[keep_column]
            new_row['date'] = current_date
            new_data.append(new_row)

    return pd.DataFrame(new_data)

In [ ]:
model_df = final_df.copy()
print(model_df.columns)
model_df.drop(['pm25_ica', 'pm10_ica', 'o3_ica', 'no2_ica', 'so2_ica',
               'co_ica', 'rrInst'], axis=1, inplace=True)


In [ ]:
# split into train and test sets
train_size = int(len(model_df) * 0.8)
test_size = len(model_df) - train_size
train, test = model_df[0:train_size], model_df[train_size:len(model_df)]
print(len(train), len(test))

train = create_interval_shifted_dataset(train, 5, 10, 'ICA')
train.dropna(inplace=True)
train.drop(['date'], axis=1, inplace=True)
print(train)




In [ ]:
#train = train.drop(train.columns[list(range(89, 107))], axis=1)
#train.columns = ['pm25(t-5)', 'pm10(t-5)', 'o3(t-5)', 'no2(t-5)', 'so2(t-5)',
#                 'co(t-5)', 'ffInst(t-5)', 'dd02Minutos(t-5)', 'ff02Minutos(t-5)',
#                 'dd10Minutos(t-5)', 'ff10Minutos(t-5)', 'hr(t-5)', 'p0(t-5)',
#                 'qfe1(t-5)', 'qfe2(t-5)', 'qff(t-5)', 'rrInst(t-5)',
#                 'radiacionGlobalInst(t-5)', 'ICA']

# Normalizar los datos
scalerX = MinMaxScaler(feature_range=(0, 1))
scalerY = MinMaxScaler(feature_range=(0, 1))
#train = scalerTrain.fit_transform(train)
train = train.values

train_x, train_y = train[:, :-1], train[:, -1]

train_x = scalerX.fit_transform(train_x)
train_y = scalerY.fit_transform(train_y.reshape(-1, 1))

#test = test.drop(test.columns[list(range(89, 107))], axis=1)
#test.columns = ['pm25(t-5)', 'pm10(t-5)', 'o3(t-5)', 'no2(t-5)', 'so2(t-5)',
#               'co(t-5)', 'ffInst(t-5)', 'dd02Minutos(t-5)', 'ff02Minutos(t-5)',
#               'dd10Minutos(t-5)', 'ff10Minutos(t-5)', 'hr(t-5)', 'p0(t-5)',
#              'qfe1(t-5)', 'qfe2(t-5)', 'qff(t-5)', 'rrInst(t-5)',
#             'radiacionGlobalInst(t-5)', 'ICA']

test = create_interval_shifted_dataset(test, 5, 10, 'ICA')
test.drop(['date'], axis=1, inplace=True)
test.dropna(inplace=True)
test = test.values
test_x, test_y = test[:, :-1], test[:, -1]
test_x = scalerX.fit_transform(test_x)
test_y = scalerY.fit_transform(test_y.reshape(-1, 1))
print(test.shape)

#test = test.values


In [ ]:
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
test_x = test_x.reshape((test_x.shape[0], train_x.shape[1], 1))
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
print(train_x.shape[1], train_x.shape[2])

# Graficar train_x y test_x
plt.figure()
plt.plot(range(train_x.shape[1]), train_x[0, :, 0], 'b-', label='Train Data')
plt.plot(range(test_x.shape[1]), test_x[0, :, 0], 'r-', label='Test Data')
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Datos de entrenamiento y prueba')
plt.legend()
plt.show()


In [ ]:
# create and fit the LSTM network
batch_size = 5
model = Sequential()
model.add(LSTM(20, batch_input_shape=(batch_size, train_x.shape[1], 1), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
from matplotlib import pyplot as plt
#pytest

history = model.fit(train_x, train_y, epochs=20, batch_size= batch_size, verbose=2, shuffle=False)
plt.plot(history.history['loss'], label='train')
plt.legend()
plt.show()

In [ ]:
# make predictions
train_predict = model.predict(train_x)
print(train_predict.shape)

model.reset_states()
test_predict = model.predict(test_x)

# Graficar train_predict y test_predict
plt.figure()
plt.plot(range(train_predict.shape[0]), train_predict[:, 0, 0], 'b-', label='Train Predictions')
plt.plot(range(test_predict.shape[0]), test_predict[:, 0, 0], 'r-', label='Test Predictions')
plt.xlabel('Muestra')
plt.ylabel('Valor')
plt.xlim([0, 250])
plt.title('Predicciones de entrenamiento y prueba')
plt.legend()
plt.show()

In [ ]:
# invert predictions
# Arreglo shatgipti
pred_prueba_y = test_predict.reshape(test_predict.shape[0], test_predict.shape[1])
pred_prueba_y = scalerX.inverse_transform(pred_prueba_y)
pred_prueba_y = pred_prueba_y[:, 0]

test_y = test_y.reshape(len(test_y), 1)
test_x2 = test_x.reshape((test_x.shape[0], test_x.shape[1]))
data_prueba_y = np.concatenate((test_y, test_x2[:, 1:]), axis=1)
data_prueba_y = scalerX.inverse_transform(data_prueba_y)
data_prueba_y = data_prueba_y[:, 0]

# Graficar los valores reales
plt.plot(data_prueba_y, label='Valores reales')

# Graficar las predicciones
plt.plot(pred_prueba_y, label='Predicciones')

plt.xlabel('df')
plt.ylabel('Variable de respuesta')
plt.title('Valores reales vs. Predicciones')
plt.legend()
plt.show()


test_score = math.sqrt(mean_squared_error(data_prueba_y, pred_prueba_y))
print('Test Score: %.2f RMSE' % (test_score))

In [ ]:

train_X = train_x.reshape((train_x.shape[0], train_x.shape[1]))
print(train_X.shape)
print(len(train_X[:, 1:]))

pred_entrenamiento_y = train_predict.reshape(train_predict.shape[0], train_predict.shape[1])
#pred_entrenamiento_y = np.concatenate((pred_entrenamiento_y, train_X[:, 1:]), axis=1)
print(pred_entrenamiento_y.shape)

pred_entrenamiento_y = scalerX.inverse_transform(pred_entrenamiento_y)
pred_entrenamiento_y = pred_entrenamiento_y[:, 0]

train_y = train_y.reshape(len(train_y), 1)
train_x2 = train_x.reshape((train_x.shape[0], train_x.shape[1]))
data_entrenamiento_y = np.concatenate((train_y, train_x2[:, 1:]), axis=1)
data_entrenamiento_y = scalerX.inverse_transform(data_entrenamiento_y)
data_entrenamiento_y = data_entrenamiento_y[:, 0]

# Graficar los valores reales
plt.plot(data_entrenamiento_y, label='Valores reales')

# Graficar las predicciones
plt.plot(pred_entrenamiento_y, label='Predicciones')

plt.xlabel('df')
plt.ylabel('Variable de respuesta')
plt.title('Valores reales vs. Predicciones')
plt.legend()
plt.show()

train_score = math.sqrt(mean_squared_error(data_entrenamiento_y, pred_entrenamiento_y))
print('Train Score: %.2f RMSE' % (train_score))

# Graficar los valores reales
plt.plot(pred_prueba_y, label='Predicción prueba')

# Graficar las predicciones
plt.plot(pred_entrenamiento_y, label='Predicción entrenamiento')

# Graficar los valores reales
plt.plot(data_prueba_y, label='Valores reales prueba')

# Graficar los valores reales entrenamiento
plt.plot(data_entrenamiento_y, label='Valores reales entrenamiento')


plt.xlabel('df')
plt.ylabel('Variable de respuesta')
plt.title('prueba vs. entrenamiento vs. reales')
plt.legend()

# Ajustar el límite del eje x
plt.xlim(0, 250)

plt.show()
